### Ref: https://huggingface.co/latent-consistency/lcm-lora-sdv1-5

In [ ]:
!pip install -q --upgrade diffusers transformers accelerate peft

# 1. Text to image

### Code run pada GPU

In [ ]:
%%time

import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "Lykon/dreamshaper-7"
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

# if Run on GPU
pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
# pipe.to("gpu")
pipe.to(device="cuda", dtype=torch.float16)

# if run on CPU
# pipe = AutoPipelineForText2Image.from_pretrained(model_id, variant="fp16")
# pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

user_prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

# disable guidance_scale by passing 0
# image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).images[0]

# Menjalankan inferensi dengan prompt yang dimasukkan pengguna
results = pipe(
    prompt=user_prompt,
    num_inference_steps=4,
    guidance_scale=0.5
    # nsfw=False
)

# Menampilkan gambar di Google Colab
display(results.images[0])

### Code run pada CPU

In [ ]:
%%time

import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "Lykon/dreamshaper-7"
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

# if Run on GPU
# pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
# pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
# pipe.to("gpu")

# if run on CPU
pipe = AutoPipelineForText2Image.from_pretrained(model_id, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

user_prompt =  "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

# disable guidance_scale by passing 0
# image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0).images[0]

# Menjalankan inferensi dengan prompt yang dimasukkan pengguna
results = pipe(
    prompt=user_prompt,
    num_inference_steps=4,
    guidance_scale=0
    # nsfw=False
)

# Menampilkan gambar di Google Colab
display(results.images[0])

## 1.a Text to image - test guidence_scale 0 - 1

In [ ]:
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image
from IPython.display import display

model_id = "Lykon/dreamshaper-7"
adapter_id = "latent-consistency/lcm-lora-sdv1-5"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# load and fuse lcm lora
pipe.load_lora_weights(adapter_id)
pipe.fuse_lora()

user_prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k" #"Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

"""
# Menjalankan serangkaian pengujian dengan berbagai nilai guidance_scale
num_tests = 11  # Jumlah pengujian, termasuk guidance_scale=0 hingga 10
images = []

for guidance_scale_value in range(num_tests):
    # Menjalankan inferensi dengan nilai guidance_scale yang berbeda
    results = pipe(
        prompt=user_prompt,
        num_inference_steps=8,
        guidance_scale=guidance_scale_value
        # nsfw=False
    )

    # Menyimpan gambar pada list images
    images.append(results.images[0])
"""

# Menjalankan serangkaian pengujian dengan berbagai nilai guidance_scale
guidance_scale_values = [i / 10.0 for i in range(11)]  # [0.0, 0.1, 0.2, ..., 1.0]
images = []

for guidance_scale_value in guidance_scale_values:
    # Menjalankan inferensi dengan nilai guidance_scale yang berbeda
    results = pipe(
        prompt=user_prompt,
        num_inference_steps=8,
        guidance_scale=guidance_scale_value
        # nsfw=False
    )

    # Menyimpan gambar pada list images
    images.append(results.images[0])

# Menampilkan semua gambar
for i, image in enumerate(images):
    display(image)

# Show all images

In [ ]:
import matplotlib.pyplot as plt

# Menampilkan gambar dengan mengatur ukuran subplot dinamis
num_rows = (len(images) - 1) // 4 + 1
fig, axs = plt.subplots(num_rows, 4, figsize=(20, 5 * num_rows))

for i in range(num_rows):
    for j in range(4):
        index = i * 4 + j
        if index < len(images):
            axs[i, j].imshow(images[index])
            axs[i, j].axis('off')
            axs[i, j].set_title(f'Guidance Scale = {guidance_scale_values[index]:.1f}')

plt.show()

# 2. Image2Image GPU

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image, LCMScheduler
from diffusers.utils import make_image_grid, load_image

pipe = AutoPipelineForImage2Image.from_pretrained(
    "Lykon/dreamshaper-7",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

# set scheduler
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load LCM-LoRA
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
pipe.fuse_lora()

# prepare image
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"
init_image = load_image(url)
prompt = "Original face, self-portrait oil painting, a handsome cyborg with golden hair, 8k"

# pass prompt and image to pipeline
generator = torch.manual_seed(0)
image = pipe(
    prompt,
    image=init_image,
    num_inference_steps=8,
    guidance_scale=0.5,
    strength=0.6,
    generator=generator
).images[0]
make_image_grid([init_image, image], rows=1, cols=2)


# 3. Inpainting GPU

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting, LCMScheduler
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

# set scheduler
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load LCM-LoRA
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
pipe.fuse_lora()

# load base and mask image
init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

# generator = torch.Generator("cuda").manual_seed(92)
prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    generator=generator,
    num_inference_steps=4,
    guidance_scale=4,
).images[0]
make_image_grid([init_image, mask_image, image], rows=1, cols=3)


4. ControlNet GPU

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, LCMScheduler
from diffusers.utils import load_image

image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
).resize((512, 512))

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
    variant="fp16"
).to("cuda")

# set scheduler
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# load LCM-LoRA
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

generator = torch.manual_seed(0)
image = pipe(
    "the mona lisa",
    image=canny_image,
    num_inference_steps=4,
    guidance_scale=1.5,
    controlnet_conditioning_scale=0.8,
    cross_attention_kwargs={"scale": 1},
    generator=generator,
).images[0]
make_image_grid([canny_image, image], rows=1, cols=2)
